In [ ]:
import os
if not os.path.isfile("estrutura_ud.py"):
    os.system("wget https://raw.githubusercontent.com/alvelvis/ACDC-UD/master/estrutura_ud.py")
if not os.path.isfile("tokenization.py"):
    os.system("wget https://raw.githubusercontent.com/alvelvis/ACDC-UD/master/tokenization.py")
import estrutura_ud
import tokenization
import pprint
import random

file = "Petroles_3_golden.conllu"
release_file = "PetroGold.conllu"

def change_col(col, token, value):
    col = token.__dict__[col].split("|")
    col = [x for x in col if not x.startswith(value.split("=")[0]) and x != "_"] + [value]
    col = "|".join(sorted(col))
    return col

def most_common(lst):
    return max(set(lst), key=lst.count)

def count_tokens(corpus, i=0):
    for sentence in corpus.sentences.values():
        i += len([x for x in sentence.tokens if not '-' in x.id])
    return i

def count_words(corpus, i=0):
    for sentence in corpus.sentences.values():
        i += len([x for x in sentence.tokens if not '-' in x.id and x.upos != "PUNCT"])
    return i

def count_documents(corpus):
    return len(set([x.rsplit("-", 1)[0] for x in corpus.sentences]))

def count_entities(corpus, i=0):
    for sentence in corpus.sentences.values():
        i += len([x for x in sentence.tokens if 'B=' in x.deps])
    return i

def count_all(label, corpus, stats=""):
    if not stats:
        stats = {'sentences': {}, 'tokens': {}, 'entities': {}, 'documents': {}, 'words': {}}
    stats['sentences'][label] = len(corpus.sentences)
    stats['tokens'][label] = count_tokens(corpus)
    stats['documents'][label] = count_documents(corpus)
    stats['entities'][label] = count_entities(corpus)
    stats['words'][label] = count_words(corpus)
    return stats

In [ ]:
# COSMO sem root e ciclo
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

for sentence in corpus.sentences.values():
    if not any(x.deprel == "root" for x in sentence.tokens):
        sentence.tokens[0].misc = "SEM ROOT"
    elif sentence.to_str().count("\troot\t") > 1:
        for token in sentence.tokens:
            if token.deprel == "root":
                token.misc = "MAIS DE 1 ROOT"
    else:
        for token in sentence.tokens:
            if (
                token.dephead != "0" and (
                    token.dephead == token.head_token.dephead or 
                    token.dephead == token.id or 
                    token.id == token.head_token.dephead or 
                    (token.head_token.dephead != "0" and token.head_token.head_token.dephead == token.id) or
                    (token.head_token.dephead != "0" and token.head_token.head_token.dephead != "0" and token.head_token.head_token.head_token.dephead == token.id)
                )):
                token.misc = "CICLO"
            if token.dephead == "0" and token.deprel != "root":
                token.misc = "root"
            if token.dephead != "0" and token.deprel == "root":
                token.misc = "root"

file_name = file.rsplit("/", 1)[1] if '/' in file else file
corpus.save("regras.conllu")
os.system("python3 ../../conllu-merge-resolver/cosmo.py {} regras.conllu '.*'".format(file))
os.system("cp {} antes-{}".format(file, file_name))
os.system("rm regras-correcao.zip; zip regras-correcao.zip antes-{} regras.conllu".format(file_name))
os.remove("antes-{}".format(file_name))
os.remove("regras.conllu")

In [ ]:
# correções de formato: ordem alfabética das feats, =, e deprels que não podem ter filho
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

for sentence in corpus.sentences.values():
    for token in sentence.tokens:
        if '</' in token.to_str():
            token.misc = "</"
        if ' ' in token.to_str():
            token.misc = "espaço em branco"
        if not token.feats.strip():
            token.feats = "_"
        if token.feats != "_":
            try:
                parts = [x.split('=') for x in token.feats.split("|")]
                parts = dict(parts)
            except ValueError:
                token.misc = "feats com erro"
            token.feats = "|".join(sorted(token.feats.split("|"), key=lambda x: x.lower()))
    
    cant_have_children = "flat:name compound punct fixed aux cop case mark punct".split()
    for token in sentence.tokens:
        if token.deprel not in "fixed compound".split() and token.head_token.deprel in cant_have_children:
            token.dephead = token.head_token.dephead if token.head_token.head_token.deprel not in cant_have_children else token.head_token.head_token.dephead

file_name = file.rsplit("/", 1)[1] if '/' in file else file
corpus.save("regras.conllu")
os.system("python3 ../../conllu-merge-resolver/cosmo.py {} regras.conllu '.*'".format(file))
os.system("cp {} antes-{}".format(file, file_name))
os.system("rm regras-correcao.zip; zip regras-correcao.zip antes-{} regras.conllu".format(file_name))
os.remove("antes-{}".format(file_name))
os.remove("regras.conllu")

In [ ]:
# text contrações
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

changed_sentences = []
for sentence in corpus.sentences.values():
    if not sentence.text.endswith(sentence.tokens[-1].word):
        contractions = [[int(x.id.split("-")[0]), int(x.id.split("-")[1]), x.word] for x in sentence.tokens if '-' in x.id]
        new_text = [x.word for x in sentence.tokens if not '-' in x.id]
        for contraction in reversed(contractions):
            start = contraction[0]
            end = contraction[1]
            word = contraction[2]
            for i in reversed(range(start-1, end)):
                new_text.pop(i)
            new_text.insert(start-1, word)
        new_text = " ".join(new_text)
        sentence.metadados['text'] = new_text
        changed_sentences.append(sentence.sent_id)

print("|".join(changed_sentences))

corpus.save("regras.conllu")
if changed_sentences:
    os.system("meld --diff {} regras.conllu".format(file))

In [ ]:
# eliminando pontuação duplicada
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

changed_sentences = []
for sentid, sentence in corpus.sentences.items():
    if sentence.tokens[-1].word == "." and sentence.tokens[-2].word == ".":
        sentence = tokenization.addToken(corpus, sentid, 'rm', sentence.tokens[-1].id).sentences[sentid]
        sentence.tokens[-1].dephead = [x for x in sentence.tokens if x.deprel == "root"][0].id
        changed_sentences.append(sentence.sent_id)
    if sentence.tokens[-1].word == "..":
        sentence.tokens[-1].word = "."
        sentence.tokens[-1].lemma = "."
        sentence.tokens[-1].dephead = [x for x in sentence.tokens if x.deprel == "root"][0].id
        changed_sentences.append(sentence.sent_id)
    if sentence.text.endswith(".."):
        sentence.metadados['text'] = sentence.text[:-1]
        changed_sentences.append(sentence.sent_id)

print("|".join(changed_sentences))

corpus.save("regras.conllu")
if changed_sentences:
    os.system("meld --diff {} regras.conllu".format(file))

In [ ]:
# tokens com ..
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

for sentid, sentence in corpus.sentences.items():
    if sentence.tokens[-1].word.endswith("..") and (sentence.tokens[-1].upos == "PUNCT" or sentence.tokens[-1].deprel == "punct"):
        sentence.tokens[-1].upos = "EPA"
        sentence.tokens[-1].misc = sentence.tokens[-1].misc.replace("tokenização", "").replace("||", "|")
        if not sentence.tokens[-1].misc:
            sentence.tokens[-1].misc = "_"

corpus.save("regras.conllu")
os.system("python3 ../../conllu-merge-resolver/cosmo.py {} regras.conllu '.*'".format(file))
os.system("cp {} antes-{}".format(file, file_name))
os.system("rm regras-correcao.zip; zip regras-correcao.zip antes-{} regras.conllu".format(file_name))
os.remove("antes-{}".format(file_name))
os.remove("regras.conllu")

In [ ]:
# tokens com .. parte 2
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

changed_sentences = []
for sentid, sentence in corpus.sentences.items():
    if sentence.tokens[-1].word.endswith(".."):
        sentence = tokenization.addToken(corpus, sentid, 'add', str(int(sentence.tokens[-1].id)+1), new_tokens=[]).sentences[sentid]
        sentence.tokens[-1].dephead = [x for x in sentence.tokens if x.deprel == "root"][0].id
        sentence.tokens[-1].lemma = "."
        sentence.tokens[-1].word = "."
        sentence.tokens[-1].deprel = "punct"
        sentence.tokens[-1].upos = "PUNCT"
        sentence.tokens[-2].word = sentence.tokens[-2].word[:-2]
        sentence.tokens[-2].lemma = sentence.tokens[-2].lemma[:-2]
        changed_sentences.append(sentence.sent_id)

print("|".join(changed_sentences))

corpus.save("regras.conllu")
if changed_sentences:
    os.system("meld --diff {} regras.conllu".format(file))

In [ ]:
# eliminando frases
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

old_sentences = set(corpus.sentences.keys())
corpus.sentences = {x: corpus.sentences[x] for x in corpus.sentences if not x.startswith("258-")}
corpus.sentences = {x: corpus.sentences[x] for x in corpus.sentences if not any(y in corpus.sentences[x].to_str() for y in "sentenciação".split())}
corpus.sentences = {x: corpus.sentences[x] for x in corpus.sentences if not any(y in corpus.sentences[x].to_str() for y in "tokenização".split())}
corpus.sentences = {x: corpus.sentences[x] for x in corpus.sentences if not any(y in corpus.sentences[x].to_str() for y in "eliminar".split())}
new_sentences = set(corpus.sentences.keys())

if new_sentences != old_sentences:
    print(len(old_sentences))
    print(len(new_sentences))

corpus.save("regras.conllu")

if new_sentences != old_sentences:
    os.system("meld --diff {} regras.conllu".format(file))

In [ ]:
# VALIDATE.PY => erros de formato devem ser corrigidos
os.system('~/Documents/Interrogat-rio/www/cgi-bin/tools/validate.py {} --lang=en'.format(file))

In [ ]:
# REMEMBER TO COMMIT BEFORE THIS
# list sentences removed
pre_commit = "5f512e8986b79be994e2c89787086b670aa7089d"
pos_commit = "master"

corpus = os.popen("git show {}:Petroles_3/{}".format(pre_commit, file)).read()
corpus_pre = estrutura_ud.Corpus(recursivo=True)
corpus_pre.build(corpus)

corpus = os.popen("git show {}:Petroles_3/{}".format(pos_commit, file)).read()
corpus_pos = estrutura_ud.Corpus(recursivo=True)
corpus_pos.build(corpus)

stats = count_all("pré", corpus_pre)
stats = count_all("pós", corpus_pos, stats)

sent_ids = set(corpus_pre.sentences)
sent_ids_removed = sent_ids - set(corpus_pos.sentences)
print("|".join(sent_ids_removed))

pprint.pprint(stats)

In [ ]:
# remover sema, xpos, misc, nmod:appos (só para release)
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

experimento_obl_arg = "experimento-obl-arg.conllu" #False

for sentid, sentence in corpus.sentences.items():
    for token in sentence.tokens:
        token.deps = "_"
        token.xpos = "_"
        token.misc = "_"
        if token.deprel == "nmod:appos" and token.upos == "NUM":
            token.deprel = "nummod"
        elif token.deprel == "nmod:appos":
            token.deprel = "nmod"
        if token.deprel == "expl:impers":
            token.deprel = "expl"

corpus.save(release_file)

s_obl_arg = 0
t_obl_arg = 0
if experimento_obl_arg:
    for sentid, sentence in corpus.sentences.items():
        if 'obl:arg' in sentence.to_str():
            s_obl_arg += 1
        for token in sentence.tokens:
            if token.deprel == "obl:arg":
                token.deprel = "obl"
                t_obl_arg += 1
    corpus.save(experimento_obl_arg)
    print("Tokens com obl:arg convertidos: {}\nFrases com obl:arg: {}".format(t_obl_arg, s_obl_arg))

In [ ]:
# SEPARAR TRAIN E TEST
copy_sentences_from = "PetroGold-v1/petrogold" # or False if random

corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(release_file)

if experimento_obl_arg:
    obl_arg = estrutura_ud.Corpus(recursivo=True)
    obl_arg.load(experimento_obl_arg)

sentences = set(corpus.sentences)
partitions = {}
if not copy_sentences_from:
    shuffled = list(corpus.sentences)
    random.shuffle(shuffled)
    partitions['test'] = set(shuffled[:round(0.05*len(sentences))])
    partitions['dev'] = set([x for x in shuffled if x not in partitions['test']][:round(0.05*len(sentences))])
    partitions['train'] = sentences - partitions['test'] - partitions['dev']
else:
    for kind in ["train", "dev", "test"]:
        model = estrutura_ud.Corpus(recursivo=True)
        model.load("{}-{}.conllu".format(copy_sentences_from, kind))
        partitions[kind] = set(model.sentences)

assert not partitions['train'].intersection(partitions['test'])
assert not partitions['train'].intersection(partitions['dev'])
assert not partitions['test'].intersection(partitions['dev'])

train = []
test = []
dev = []
for sentid, sentence in corpus.sentences.items():
    if sentid in partitions['test']:
        test.append(sentence.to_str())
    if sentid in partitions['train']:
        train.append(sentence.to_str())
    if sentid in partitions['dev']:
        dev.append(sentence.to_str())

if experimento_obl_arg:
    train_obl_arg = []
    test_obl_arg = []
    dev_obl_arg = []
    for sentid, sentence in obl_arg.sentences.items():
        if sentid in partitions['test']:
            test_obl_arg.append(sentence.to_str())
        if sentid in partitions['train']:
            train_obl_arg.append(sentence.to_str())
        if sentid in partitions['dev']:
            dev_obl_arg.append(sentence.to_str())

corpus_train = estrutura_ud.Corpus(recursivo=True)
corpus_test = estrutura_ud.Corpus(recursivo=True)
corpus_dev = estrutura_ud.Corpus(recursivo=True)
corpus_train.build("\n\n".join(train) + "\n\n")
corpus_test.build("\n\n".join(test) + "\n\n")
corpus_dev.build("\n\n".join(dev) + "\n\n")

stats = count_all("train", corpus_train)
stats = count_all("test", corpus_test, stats)
stats = count_all("dev", corpus_dev, stats)
pprint.pprint(stats)

corpus_train.save("petrogold-train.conllu")
corpus_test.save("petrogold-test.conllu")
corpus_dev.save("petrogold-dev.conllu")

if experimento_obl_arg:
    print("Experimento obl:arg:")
    corpus_train = estrutura_ud.Corpus(recursivo=True)
    corpus_test = estrutura_ud.Corpus(recursivo=True)
    corpus_dev = estrutura_ud.Corpus(recursivo=True)
    corpus_train.build("\n\n".join(train_obl_arg) + "\n\n")
    corpus_test.build("\n\n".join(test_obl_arg) + "\n\n")
    corpus_dev.build("\n\n".join(dev_obl_arg) + "\n\n")

    stats = count_all("train", corpus_train)
    stats = count_all("test", corpus_test, stats)
    stats = count_all("dev", corpus_dev, stats)
    pprint.pprint(stats)

    corpus_train.save("experimento-obl-arg-train.conllu")
    corpus_test.save("experimento-obl-arg-test.conllu")
    corpus_dev.save("experimento-obl-arg-dev.conllu")

In [ ]:
# TREINAR COM TRAIN E DEV
! ./udpipe-1.2.0 --train --tokenizer=none "petrogold.udpipe" "petrogold-train.conllu" "petrogold-dev.conllu"
! ./udpipe-1.2.0 --train --tokenizer=none "experimento-obl-arg.udpipe" "experimento-obl-arg-train.conllu" "experimento-obl-arg-dev.conllu"
#! ./udpipe-1.2.0 --train --tokenizer=none "bosque-ud-2.8/bosque2.8.udpipe" "bosque-ud-2.8/pt_bosque-ud-train.conllu" "bosque-ud-2.8/pt_bosque-ud-dev.conllu"

In [ ]:
# EVALUATE PETRO3
! python3 ~/Documents/ACDC-UD/tokenizar_conllu.py "petrogold-test.conllu" "petrogold-test-tokenizado.txt"
! python3 ~/Documents/ACDC-UD/udpipe_vertical.py "petrogold.udpipe" "petrogold-test-tokenizado.txt" "petrogold-test-udpipe.conllu"
! python3 ~/Documents/ACDC-UD/conll18_ud_eval.py -v "petrogold-test.conllu" "petrogold-test-udpipe.conllu"

In [ ]:
# EVALUATE PETRO3 EXPERIMENTO OBL:ARG
! python3 ~/Documents/ACDC-UD/tokenizar_conllu.py "experimento-obl-arg-test.conllu" "experimento-obl-arg-test-tokenizado.txt"
! python3 ~/Documents/ACDC-UD/udpipe_vertical.py "experimento-obl-arg.udpipe" "experimento-obl-arg-test-tokenizado.txt" "experimento-obl-arg-test-udpipe.conllu"
! python3 ~/Documents/ACDC-UD/conll18_ud_eval.py -v "experimento-obl-arg-test.conllu" "experimento-obl-arg-test-udpipe.conllu"

In [ ]:
# EVALUATE BOSQUE2.8
! python3 ~/Documents/ACDC-UD/tokenizar_conllu.py "bosque-ud-2.8/pt_bosque-ud-test.conllu" "bosque-ud-2.8/pt_bosque-ud-test-tokenizado.txt"
! python3 ~/Documents/ACDC-UD/udpipe_vertical.py "bosque-ud-2.8/bosque2.8.udpipe" "bosque-ud-2.8/pt_bosque-ud-test-tokenizado.txt" "bosque-ud-2.8/pt_bosque-ud-test-udpipe.conllu"
! python3 ~/Documents/ACDC-UD/conll18_ud_eval.py -v "bosque-ud-2.8/pt_bosque-ud-test.conllu" "bosque-ud-2.8/pt_bosque-ud-test-udpipe.conllu"

In [ ]:
# COUNT MODIFICATIONS (usado só no STIL)
file = "Petroles_3_golden.conllu"
file_stanza = "Petroles_3_stanza.conllu"
file_udpipe = "Petroles_3_udpipe.conllu"
post_changes = "master"

corpus = os.popen("git show {}:Petroles_3/{}".format(post_changes, file)).read()
corpus_pos = estrutura_ud.Corpus(recursivo=True)
corpus_pos.build(corpus)

corpus_stanza = estrutura_ud.Corpus(recursivo=True)
corpus_stanza.load(file_stanza)
corpus_udpipe = estrutura_ud.Corpus(recursivo=True)
corpus_udpipe.load(file_udpipe)

sentences_that_matter = list(corpus_pos.sentences.keys())
print("Sentences that matter: {}".format(len(sentences_that_matter)))

tokens_changed = set()
tokens_changed_lemma = set()
tokens_changed_deprel = set()
tokens_changed_dephead = set()
tokens_changed_upos = set()
sentences_changed = set()
udpipe_correct = set()
all_tokens = 0
all_sentences = 0
cm = set()
for sentid in sentences_that_matter:
    for t, token in enumerate(corpus_pos.sentences[sentid].tokens):
        if not '-' in token.id and token.upos != "PUNCT":
            all_tokens += 1
            try:
                token_pos = corpus_pos.sentences[sentid].tokens[t]
                token_stanza = corpus_stanza.sentences[sentid].tokens[t]
                token_udpipe = corpus_udpipe.sentences[sentid].tokens[t]
                important_cols_pos = [token_pos.lemma, token_pos.upos, token_pos.feats, token_pos.dephead, token_pos.deprel]
                important_cols_stanza = [token_stanza.lemma, token_stanza.upos, token_stanza.feats, token_stanza.dephead, token_stanza.deprel]
                important_cols_udpipe = [token_udpipe.lemma, token_udpipe.upos, token_udpipe.feats, token_udpipe.dephead, token_udpipe.deprel]
                if token_stanza.deprel != token_udpipe.deprel:
                    cm.add("{}-{}".format(sentid, t))
                    if token_udpipe.deprel == token_pos.deprel:
                        udpipe_correct.add("{}-{}".format(sentid, t))
                if important_cols_pos != important_cols_stanza:
                    #if token_pos.deprel == "obl:arg" and token_stanza.deprel in ["obj", "iobj"]:
                        #continue
                    tokens_changed.add("{}-{}".format(sentid, t))
                    if not sentid in sentences_changed:
                        sentences_changed.add(sentid)
                    if token_pos.lemma != token_stanza.lemma:
                        tokens_changed_lemma.add("{}-{}".format(sentid, t))
                    if token_pos.upos != token_stanza.upos:
                        tokens_changed_upos.add("{}-{}".format(sentid, t))
                    if token_pos.dephead != token_stanza.dephead:
                        tokens_changed_dephead.add("{}-{}".format(sentid, t))
                    if token_pos.deprel != token_stanza.deprel:
                        tokens_changed_deprel.add("{}-{}".format(sentid, t))
            except:
                print("token out of index")

In [ ]:
# DISPLAY INTERSEÇÕES (usado só no STIL)
print("EXCEPT PUNCT -- Tokens changed: {} / All_tokens: {}".format(len(tokens_changed), all_tokens))
print("Sentences changed: {} / All_sentences: {}".format(len(sentences_changed), len(sentences_that_matter)))
print("CM: {} / Intersection: {} / UDPIPE_CORRECT: {}".format(len(cm), len(tokens_changed.intersection(cm)), len(udpipe_correct)))
print(len(tokens_changed_lemma))
print(len(tokens_changed_upos))
print(len(tokens_changed_dephead))
print(len(tokens_changed_deprel))
print("")
print(len(tokens_changed_lemma.intersection(cm)))
print(len(tokens_changed_upos.intersection(cm)))
print(len(tokens_changed_dephead.intersection(cm)))
print(len(tokens_changed_deprel.intersection(cm)))
print("")
print(len(tokens_changed_lemma.intersection(tokens_changed_upos)))
print(len(tokens_changed_lemma.intersection(tokens_changed_upos).intersection(cm)))
print(len(tokens_changed_lemma.intersection(tokens_changed_deprel)))
print(len(tokens_changed_lemma.intersection(tokens_changed_deprel).intersection(cm)))
print(len(tokens_changed_upos.intersection(tokens_changed_deprel)))
print(len(tokens_changed_upos.intersection(tokens_changed_deprel).intersection(cm)))
print(len(tokens_changed_dephead.intersection(tokens_changed_deprel)))
print(len(tokens_changed_dephead.intersection(tokens_changed_deprel).intersection(cm)))
print(len(tokens_changed_lemma - tokens_changed_upos - tokens_changed_dephead - tokens_changed_deprel))